In [1]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import re
import numpy as np
import os
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
###importing the data and checking through for anormalies 
data2020 = pd.read_csv('2020b table uncleaned -Copy1.csv', parse_dates=['date'])
data2020.tail(50)
data2020.isnull().sum()
data2020.shape[0]

/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


528469

In [3]:
##creating a check point and droping the first rows##
####
data2020.columns
##the addition below is very important in other to avioid loosing data from the other colums with nan
data2020['dimension']= data2020['pagePath'].astype(str) + data2020['dimension36'].astype(str)### 

In [4]:
relevant = ['viewid','date','pagePath','hits','dimension']
data_to_clean = data2020[relevant]
data_to_clean.isnull().sum()

viewid       0
date         0
pagePath     0
hits         0
dimension    0
dtype: int64

In [5]:
##replacing url column over with data from pagepath 
#data2019.dimension36.fillna(data2019.pagePath, inplace=True) ##replacing data in the columns (dimension url with pagePath data)

##run check##
#data2020.isnull().sum()

creating new column to group new model from old model

In [6]:
##creating function for to find out whether car is new or new or old (Newness Type) in pagePath# ##
data_to_clean["newness_type"]=data_to_clean["dimension"].str.extract('(new|used)')
data_to_clean['newness_type'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type'].unique() ##check unique values 
data_to_clean['newness_type'].value_counts() ##count unique values 

<ipython-input-6-1b06e7829788>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean["newness_type"]=data_to_clean["dimension"].str.extract('(new|used)')
/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


unknown_type    499303
new              24250
used              4916
Name: newness_type, dtype: int64

In [7]:
##creating function for to find out whether car is new or new or old in Dimension36 (As Newness Type) ##
data_to_clean["newness_type2"]=data_to_clean["dimension"].str.extract('(new|used)')
data_to_clean['newness_type2'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type2'].unique() ##check unique values 
data_to_clean['newness_type2'].value_counts() ##count unique values 

<ipython-input-7-218694c3b25b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean["newness_type2"]=data_to_clean["dimension"].str.extract('(new|used)')


unknown_type    499303
new              24250
used              4916
Name: newness_type2, dtype: int64

In [8]:
###viewing data without unknown newness type##
data_to_clean[data_to_clean['newness_type']== 'unknown_type'].head(20)

,viewid,date,pagePath,hits,dimension,newness_type,newness_type2
0,129875374,2020-12-08,/acura/ilx/,1,/acura/ilx/nan,unknown_type,unknown_type
1,129875374,2020-12-08,/acura/ilx/2019/st-401776135/features-specs/,1,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type,unknown_type
2,129875374,2020-12-08,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type,unknown_type
3,129875374,2020-12-08,/acura/mdx/,8,/acura/mdx/nan,unknown_type,unknown_type
4,129875374,2020-12-08,/acura/mdx/2017/vin/5j8yd4h51hl004844/,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type,unknown_type
5,129875374,2020-12-08,/acura/mdx/2018/vin/5j8yd4h58jl002742/,1,/acura/mdx/2018/vin/5j8yd4h58jl002742/nan,unknown_type,unknown_type
6,129875374,2020-12-08,/acura/mdx/2018/vin/5j8yd4h58jl002742/?radius=500,1,/acura/mdx/2018/vin/5j8yd4h58jl002742/?radius=...,unknown_type,unknown_type
7,129875374,2020-12-08,/acura/mdx/2019/vin/5j8yd3h33kl008720/?radius=100,1,/acura/mdx/2019/vin/5j8yd3h33kl008720/?radius=...,unknown_type,unknown_type
8,129875374,2020-12-08,/acura/mdx/2019/vin/5j8yd3h35kl011408/?radius=100,1,/acura/mdx/2019/vin/5j8yd3h35kl011408/?radius=...,unknown_type,unknown_type
9,129875374,2020-12-08,/acura/mdx/build/?submodel=suv&modelyearid=401...,1,/acura/mdx/build/?submodel=suv&modelyearid=401...,unknown_type,unknown_type


In [9]:
data_to_clean['newness_type'].unique()
data_to_clean['newness_type'].value_counts()

unknown_type    499303
new              24250
used              4916
Name: newness_type, dtype: int64

In [10]:
check = ["newness_type2","newness_type"]
data_to_clean[check]
#data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,newness_type2,newness_type
0,unknown_type,unknown_type
1,unknown_type,unknown_type
2,unknown_type,unknown_type
3,unknown_type,unknown_type
4,unknown_type,unknown_type
...,...,...
528464,unknown_type,unknown_type
528465,unknown_type,unknown_type
528466,unknown_type,unknown_type
528467,unknown_type,unknown_type


In [11]:
data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,viewid,date,pagePath,hits,dimension,newness_type,newness_type2


In [12]:
### merging btoh newness column to have one column## 
data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2


##Drop  Newness_type2##
##clean_1 = data_to_clean[data_to_clean.newness_type!= 'unknown_type']
clean_1 = data_to_clean.drop(['newness_type2'], axis=1)##break point after removing newness_type2 data  or newness or use type

###clean_1.drop(['newness_type2'], axis=1, inplace=True) ##code if if I have to remove the unknown newness type 


clean_1.shape[0]

<ipython-input-12-f23abb027719>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2
/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)


528469

In [13]:
data_to_clean['newness_type'].value_counts()

unknown_type    499303
new              24250
used              4916
Name: newness_type, dtype: int64

In [14]:
##data_to_clean[data_to_clean['newness_type']== 'unknown_type'].tail(100)

In [15]:
clean_1.head() ##previewing data 


,viewid,date,pagePath,hits,dimension,newness_type
0,129875374,2020-12-08,/acura/ilx/,1,/acura/ilx/nan,unknown_type
1,129875374,2020-12-08,/acura/ilx/2019/st-401776135/features-specs/,1,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type
2,129875374,2020-12-08,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type
3,129875374,2020-12-08,/acura/mdx/,8,/acura/mdx/nan,unknown_type
4,129875374,2020-12-08,/acura/mdx/2017/vin/5j8yd4h51hl004844/,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type


In [16]:
#####working on the url (dimension36) too extract importanst data#
###first removing the url beginning ###
##removing the beginning url for consistencies in the column 


In [17]:
clean_2= clean_1.copy()
clean_2

,viewid,date,pagePath,hits,dimension,newness_type
0,129875374,2020-12-08,/acura/ilx/,1,/acura/ilx/nan,unknown_type
1,129875374,2020-12-08,/acura/ilx/2019/st-401776135/features-specs/,1,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type
2,129875374,2020-12-08,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type
3,129875374,2020-12-08,/acura/mdx/,8,/acura/mdx/nan,unknown_type
4,129875374,2020-12-08,/acura/mdx/2017/vin/5j8yd4h51hl004844/,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type
...,...,...,...,...,...,...
528464,129875374,2020-04-02,/land-rover/range-rover-evoque/,1,/land-rover/range-rover-evoque/nan,unknown_type
528465,129875374,2020-04-02,/land-rover/range-rover-evoque/2015/vin/salvn2...,1,/land-rover/range-rover-evoque/2015/vin/salvn2...,unknown_type
528466,129875374,2020-04-02,/land-rover/range-rover-sport/,1,/land-rover/range-rover-sport/nan,unknown_type
528467,129875374,2020-04-02,/land-rover/range-rover-sport/2019/vin/salwr2r...,1,/land-rover/range-rover-sport/2019/vin/salwr2r...,unknown_type


In [18]:
####Extracting Values for make, model and year

###first create another variable "url" by modifying "variable dimension36" which will have a spacebar the end for uniformity ##
clean_2['url'] = clean_2['dimension'].astype(str) + ' '

####Extracting Values for make, model and year
clean_2['Maker']=clean_2['dimension'].str.extract('(make=[\w+-]+\W)')
clean_2['Makerb']=clean_2['url'].str.extract('(make=[a-zA-Z0-9_\%\W]+ $)')
clean_2['Model']=clean_2['dimension'].str.extract('(model=[\w+-]+\W)')
clean_2['Modelb']=clean_2['url'].str.extract('(model=[a-zA-Z0-9_\%\W]+ $)')
clean_2['Year']=clean_2['dimension'].str.extract('(year=......)')
clean_2['Extcolor']=clean_2['url'].str.extract('(extcolor=[\w+]+\W)')
clean_2['Intcolor']=clean_2['url'].str.extract('(intcolor=[\w+]+\W)')
####clean_2['ModelB']=clean_2['url'].str.extract('(model=[a-zA-Z0-9_\-]+ $)')


##replace the separator % with & for uniformity##
clean_2['Maker']=clean_2['Maker'].str.replace('%','&')
clean_2['Model']=clean_2['Model'].str.replace('%','&')
clean_2['Year']=clean_2['Year'].str.replace('%','&')
#clean_2['Modelb']=clean_2['Modelb'].str.replace(' ','&')###not necessary because we are splitting on spacebar
#clean_2['Makerb']=clean_2['Makerb'].str.replace(' ','&')###not necessary because we are splitting on spacebar



##Fill all Null values within columns make, model and year####
clean_2['Maker'].fillna('unknown', inplace=True)
clean_2['Makerb'].fillna('unknown', inplace=True)
clean_2['Model'].fillna('unknown', inplace=True)
clean_2['Modelb'].fillna('unknown', inplace=True)
clean_2['Year'].fillna('unknown', inplace=True)
clean_2['Extcolor'].fillna('unknown', inplace=True)
clean_2['Intcolor'].fillna('unknown', inplace=True)


###creating a cleaned column for variable make, model and yea## 
clean_2['OEM']=clean_2['Maker'].apply(lambda st: st[st.find("make=")+5:st.find("\W")])
clean_2['OEMB']=clean_2['Makerb'].apply(lambda st: st[st.find("make=")+5:st.find(" ")])
clean_2['MODEL']=clean_2['Model'].apply(lambda st: st[st.find("model=")+6:st.find("\W")])
clean_2['MODELB']=clean_2['Modelb'].apply(lambda st: st[st.find("model=")+6:st.find(" ")])
clean_2['YEAR_BUILT']=clean_2['Year'].str.extract('(\d+)') 
##clean_2['YEAR_BUILT']=clean_2['Year'].apply(lambda st: st[st.find("year=")+5:st.find("\W")-1])
clean_2['INTCOLOR']=clean_2['Extcolor'].apply(lambda st: st[st.find("extcolor=")+9:st.find("\W")])  ######this will dates without ranges 
clean_2['EXTCOLOR']=clean_2['Intcolor'].apply(lambda st: st[st.find("intcolor=")+9:st.find("\W")]) ###this will get data for dates that have ranges ranges



##replace ow/w values within columns for clarification####
clean_2['OEM'].replace({"ow": "unknown"}, inplace=True)
clean_2['OEMB'].replace({"ow": "unknown"}, inplace=True)
clean_2['MODEL'].replace({"w": "unknown"}, inplace=True)
clean_2['MODELB'].replace({"w": "unknown"}, inplace=True)
clean_2['YEAR_BUILT'].replace({"o": "unknown"}, inplace=True)

clean_2['INTCOLOR'] = clean_2.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
clean_2['EXTCOLOR'] = clean_2.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
#clean_2['INTCOLOR'].replace({" ": "unknown"}, inplace=True)
clean_2.INTCOLOR.fillna("unknown", inplace=True)
clean_2['EXTCOLOR'].fillna("unknown", inplace=True)




###Merging both OEM and MODEL
clean_2.OEM[clean_2.OEM=='unknown']=clean_2.OEMB
clean_2.MODEL[clean_2.MODEL=='unknown']=clean_2.MODELB


##Drop unwanted columns##
####clean_2.drop(['Maker','Makerb','Model','Modelb','Year','MODELB','pagePath','url', 'OEMB'], axis=1, inplace=True)


###Needed variables 
#var= ['viewid', 'date', 'hits','dimension36','newness_type','OEM','MODEL','YEAR_BUILT']
var= ['viewid', 'date', 'hits','pagePath','dimension','newness_type','OEM','MODEL','YEAR_BUILT']
color = ['EXTCOLOR','INTCOLOR']
Data_cleaned_part1 = clean_2[var]
color_df = clean_2[color]


<ipython-input-18-87d730ec4da8>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_2.OEM[clean_2.OEM=='unknown']=clean_2.OEMB
<ipython-input-18-87d730ec4da8>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_2.MODEL[clean_2.MODEL=='unknown']=clean_2.MODELB


In [19]:
Data_cleaned_part1

,viewid,date,hits,pagePath,dimension,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2020-12-08,1,/acura/ilx/,/acura/ilx/nan,unknown_type,unknown,unknown,NaN
1,129875374,2020-12-08,1,/acura/ilx/2019/st-401776135/features-specs/,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type,unknown,unknown,NaN
2,129875374,2020-12-08,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type,unknown,unknown,NaN
3,129875374,2020-12-08,8,/acura/mdx/,/acura/mdx/nan,unknown_type,unknown,unknown,NaN
4,129875374,2020-12-08,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type,unknown,unknown,NaN
...,...,...,...,...,...,...,...,...,...
528464,129875374,2020-04-02,1,/land-rover/range-rover-evoque/,/land-rover/range-rover-evoque/nan,unknown_type,unknown,unknown,NaN
528465,129875374,2020-04-02,1,/land-rover/range-rover-evoque/2015/vin/salvn2...,/land-rover/range-rover-evoque/2015/vin/salvn2...,unknown_type,unknown,unknown,NaN
528466,129875374,2020-04-02,1,/land-rover/range-rover-sport/,/land-rover/range-rover-sport/nan,unknown_type,unknown,unknown,NaN
528467,129875374,2020-04-02,1,/land-rover/range-rover-sport/2019/vin/salwr2r...,/land-rover/range-rover-sport/2019/vin/salwr2r...,unknown_type,unknown,unknown,NaN


In [20]:
#clean_2['Modelb'].isnull().sum()
#clean_2[clean_2['Year']=='unknown']


In [21]:
clean_2['INTCOLOR'].value_counts()

unknown      525690
white           318
dark            227
black           196
platinum        151
              ...  
yas               1
cinnamon          1
nano              1
cherry            1
melbourne         1
Name: INTCOLOR, Length: 247, dtype: int64

In [22]:
###checking if theres  is color info###
clean_2[clean_2['INTCOLOR']!='']
#clean_2[8535:8550]

,viewid,date,pagePath,hits,dimension,newness_type,url,Maker,Makerb,Model,...,Year,Extcolor,Intcolor,OEM,OEMB,MODEL,MODELB,YEAR_BUILT,INTCOLOR,EXTCOLOR
0,129875374,2020-12-08,/acura/ilx/,1,/acura/ilx/nan,unknown_type,/acura/ilx/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
1,129875374,2020-12-08,/acura/ilx/2019/st-401776135/features-specs/,1,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type,/acura/ilx/2019/st-401776135/features-specs/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
2,129875374,2020-12-08,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
3,129875374,2020-12-08,/acura/mdx/,8,/acura/mdx/nan,unknown_type,/acura/mdx/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
4,129875374,2020-12-08,/acura/mdx/2017/vin/5j8yd4h51hl004844/,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528464,129875374,2020-04-02,/land-rover/range-rover-evoque/,1,/land-rover/range-rover-evoque/nan,unknown_type,/land-rover/range-rover-evoque/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
528465,129875374,2020-04-02,/land-rover/range-rover-evoque/2015/vin/salvn2...,1,/land-rover/range-rover-evoque/2015/vin/salvn2...,unknown_type,/land-rover/range-rover-evoque/2015/vin/salvn2...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
528466,129875374,2020-04-02,/land-rover/range-rover-sport/,1,/land-rover/range-rover-sport/nan,unknown_type,/land-rover/range-rover-sport/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
528467,129875374,2020-04-02,/land-rover/range-rover-sport/2019/vin/salwr2r...,1,/land-rover/range-rover-sport/2019/vin/salwr2r...,unknown_type,/land-rover/range-rover-sport/2019/vin/salwr2r...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown


In [23]:
###checking if theres  is color info###
clean_2[clean_2['MODEL']!='unknown']
#clean_2[8535:8550]

,viewid,date,pagePath,hits,dimension,newness_type,url,Maker,Makerb,Model,...,Year,Extcolor,Intcolor,OEM,OEMB,MODEL,MODELB,YEAR_BUILT,INTCOLOR,EXTCOLOR
9,129875374,2020-12-08,/acura/mdx/build/?submodel=suv&modelyearid=401...,1,/acura/mdx/build/?submodel=suv&modelyearid=401...,unknown_type,/acura/mdx/build/?submodel=suv&modelyearid=401...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401818473&styleid=401818482&op...,NaN,unknown,unknown
66,129875374,2020-12-08,/audi/q5/build/?submodel=suv&modelyearid=40184...,1,/audi/q5/build/?submodel=suv&modelyearid=40184...,unknown_type,/audi/q5/build/?submodel=suv&modelyearid=40184...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401848448&styleid=401872826&op...,NaN,unknown,unknown
74,129875374,2020-12-08,/audi/q8/build/?submodel=suv&modelyearid=40185...,1,/audi/q8/build/?submodel=suv&modelyearid=40185...,unknown_type,/audi/q8/build/?submodel=suv&modelyearid=40185...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401851133&styleid=401874136&op...,NaN,unknown,unknown
128,129875374,2020-12-08,/bmw/x3/build/?submodel=suv&modelyearid=401866...,2,/bmw/x3/build/?submodel=suv&modelyearid=401866...,unknown_type,/bmw/x3/build/?submodel=suv&modelyearid=401866...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401866210&styleid=401866230&op...,NaN,unknown,unknown
141,129875374,2020-12-08,/bmw/x7/build/?submodel=suv&modelyearid=401797...,1,/bmw/x7/build/?submodel=suv&modelyearid=401797...,unknown_type,/bmw/x7/build/?submodel=suv&modelyearid=401797...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401797268&styleid=401797270&op...,NaN,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528412,129875374,2020-04-02,/inventory/srp.html?make=subaru&model=crosstre...,1,/inventory/srp.html?make=subaru&model=crosstre...,unknown_type,/inventory/srp.html?make=subaru&model=crosstre...,make=subaru&,make=subaru&model=crosstrek&year=2019-*&option...,model=crosstrek&,...,year=2019-*,unknown,unknown,subaru,subaru&model=crosstrek&year=2019-*&options=rem...,crosstrek,crosstrek&year=2019-*&options=remote-engine-st...,2019,unknown,unknown
528413,129875374,2020-04-02,/inventory/srp.html?make=toyota&model=rav4&yea...,1,/inventory/srp.html?make=toyota&model=rav4&yea...,new,/inventory/srp.html?make=toyota&model=rav4&yea...,make=toyota&,make=toyota&model=rav4&year=2020&inventorytype...,model=rav4&,...,year=2020&i,unknown,unknown,toyota,toyota&model=rav4&year=2020&inventorytype=new&...,rav4,rav4&year=2020&inventorytype=new&trim=lenan,2020,unknown,unknown
528414,129875374,2020-04-02,/inventory/srp.html?make=toyota&model=tundra&y...,1,/inventory/srp.html?make=toyota&model=tundra&y...,unknown_type,/inventory/srp.html?make=toyota&model=tundra&y...,make=toyota&,make=toyota&model=tundra&year=2020nan,model=tundra&,...,year=2020na,unknown,unknown,toyota,toyota&model=tundra&year=2020nan,tundra,tundra&year=2020nan,2020,unknown,unknown
528415,129875374,2020-04-02,/inventory/srp.html?make=volvo&model=xc40&trim...,1,/inventory/srp.html?make=volvo&model=xc40&trim...,unknown_type,/inventory/srp.html?make=volvo&model=xc40&trim...,make=volvo&,make=volvo&model=xc40&trim=t5 r-designnan,model=xc40&,...,unknown,unknown,unknown,volvo,volvo&model=xc40&trim=t5,xc40,xc40&trim=t5,NaN,unknown,unknown


EXPORT POINT TO VIEW OUTPUT 

In [24]:
##checking for the correctness of output among variables with values 
Data_cleaned_part1.to_csv('Data_cleaned_part12019aaa.csv', encoding='utf-8')

CLEANING THE SECOND SEGMENT USING SPLIT

In [25]:
dataA = Data_cleaned_part1[Data_cleaned_part1['OEM']!='unknown'] 
dataB = Data_cleaned_part1[Data_cleaned_part1['OEM']=='unknown']

In [26]:
dataB

,viewid,date,hits,pagePath,dimension,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2020-12-08,1,/acura/ilx/,/acura/ilx/nan,unknown_type,unknown,unknown,NaN
1,129875374,2020-12-08,1,/acura/ilx/2019/st-401776135/features-specs/,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type,unknown,unknown,NaN
2,129875374,2020-12-08,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type,unknown,unknown,NaN
3,129875374,2020-12-08,8,/acura/mdx/,/acura/mdx/nan,unknown_type,unknown,unknown,NaN
4,129875374,2020-12-08,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type,unknown,unknown,NaN
...,...,...,...,...,...,...,...,...,...
528464,129875374,2020-04-02,1,/land-rover/range-rover-evoque/,/land-rover/range-rover-evoque/nan,unknown_type,unknown,unknown,NaN
528465,129875374,2020-04-02,1,/land-rover/range-rover-evoque/2015/vin/salvn2...,/land-rover/range-rover-evoque/2015/vin/salvn2...,unknown_type,unknown,unknown,NaN
528466,129875374,2020-04-02,1,/land-rover/range-rover-sport/,/land-rover/range-rover-sport/nan,unknown_type,unknown,unknown,NaN
528467,129875374,2020-04-02,1,/land-rover/range-rover-sport/2019/vin/salwr2r...,/land-rover/range-rover-sport/2019/vin/salwr2r...,unknown_type,unknown,unknown,NaN


In [27]:
##Creating checkpoint##
###CHeckpoint###
#dataA[1510:1531]
#dataA.to_csv('dataA2020vv.csv',encoding='utf-8')
#dataB.to_csv('dataB2020cc.csv',encoding='utf-8')

var = ['viewid','date','hits','pagePath','dimension','newness_type'] 
to_clean = dataB[var]
to_clean


,viewid,date,hits,pagePath,dimension,newness_type
0,129875374,2020-12-08,1,/acura/ilx/,/acura/ilx/nan,unknown_type
1,129875374,2020-12-08,1,/acura/ilx/2019/st-401776135/features-specs/,/acura/ilx/2019/st-401776135/features-specs/nan,unknown_type
2,129875374,2020-12-08,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type
3,129875374,2020-12-08,8,/acura/mdx/,/acura/mdx/nan,unknown_type
4,129875374,2020-12-08,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type
...,...,...,...,...,...,...
528464,129875374,2020-04-02,1,/land-rover/range-rover-evoque/,/land-rover/range-rover-evoque/nan,unknown_type
528465,129875374,2020-04-02,1,/land-rover/range-rover-evoque/2015/vin/salvn2...,/land-rover/range-rover-evoque/2015/vin/salvn2...,unknown_type
528466,129875374,2020-04-02,1,/land-rover/range-rover-sport/,/land-rover/range-rover-sport/nan,unknown_type
528467,129875374,2020-04-02,1,/land-rover/range-rover-sport/2019/vin/salwr2r...,/land-rover/range-rover-sport/2019/vin/salwr2r...,unknown_type


In [52]:
###first removing the beginning url string if any## 
to_clean["dimension"].replace(regex=['(http+[\W+\w+]+.edmunds.com)'], value=[''], inplace = True) ##removing the beginning url for consistencies in the column 
to_clean["dimension"].replace(regex=['(/+used+-)'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 
#to_clean["dimension36"].replace(regex=['(\A/+0-9+/)'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 
to_clean["dimension"].replace(regex=['-'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 



##splitting the url by (/)
url = to_clean["dimension"].str.split('/', expand=True)

##naming the split columns ##
url.rename(columns={0:"empty", 1:"OEM", 2:"MODEL", 3:"YEAR_BUILT", 4:"design", 5:"exitpage1", 6:"exitpage2"}, inplace=True)

##dropping unwanted columns 
neededcolumn = ['OEM','MODEL','YEAR_BUILT'] 
useful = url[neededcolumn]

/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [53]:
useful

,OEM,MODEL,YEAR_BUILT
0,acura,ilx,nan
1,acura,ilx,2019
2,acura,ilx,2021
3,acura,mdx,nan
4,acura,mdx,2017
...,...,...,...
528464,land,rover,range
528465,land,rover,range
528466,land,rover,range
528467,land,rover,range


In [54]:
##Joining the dataframe back to ##
cleaned = pd.concat([to_clean, useful], axis=1)

In [55]:

cleaned 

,viewid,date,hits,pagePath,dimension,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2020-12-08,1,/acura/ilx/,/acura/ilx/nan,unknown_type,acura,ilx,nan
1,129875374,2020-12-08,1,/acura/ilx/2019/st-401776135/features-specs/,/acura/ilx/2019/st/401776135/features/specs/nan,unknown_type,acura,ilx,2019
2,129875374,2020-12-08,1,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,/acura/ilx/2021/quickquotes.html?zip=07029&epi...,unknown_type,acura,ilx,2021
3,129875374,2020-12-08,8,/acura/mdx/,/acura/mdx/nan,unknown_type,acura,mdx,nan
4,129875374,2020-12-08,1,/acura/mdx/2017/vin/5j8yd4h51hl004844/,/acura/mdx/2017/vin/5j8yd4h51hl004844/nan,unknown_type,acura,mdx,2017
...,...,...,...,...,...,...,...,...,...
528464,129875374,2020-04-02,1,/land-rover/range-rover-evoque/,/land/rover/range/rover/evoque/nan,unknown_type,land,rover,range
528465,129875374,2020-04-02,1,/land-rover/range-rover-evoque/2015/vin/salvn2...,/land/rover/range/rover/evoque/2015/vin/salvn2...,unknown_type,land,rover,range
528466,129875374,2020-04-02,1,/land-rover/range-rover-sport/,/land/rover/range/rover/sport/nan,unknown_type,land,rover,range
528467,129875374,2020-04-02,1,/land-rover/range-rover-sport/2019/vin/salwr2r...,/land/rover/range/rover/sport/2019/vin/salwr2r...,unknown_type,land,rover,range


In [56]:
###checkpoint to view output in excel##
#cleaned.to_csv('cleaned.csv',encoding='utf-8')

In [57]:
##Fill all Null values within columns make, model and year####
cleaned['OEM'].fillna('unknown', inplace=True)
cleaned['MODEL'].fillna('unknown', inplace=True)
#cleaned['YEAR_BUILT'].fillna('unknown', inplace=True)

In [58]:
####Next is joining the data together as one ##
joined_clean = cleaned.append(dataA)

####Next is joining the data together as one ##
joined_clean= cleaned.append(dataA)


##joining the color dataframe from the original dataframe
data_with_color = pd.concat([joined_clean, color_df], axis=1)

##clean color again##
data_with_color['INTCOLOR'] = data_with_color.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
data_with_color['EXTCOLOR'] = data_with_color.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
data_with_color.INTCOLOR.fillna("unknown", inplace=True)
data_with_color['EXTCOLOR'].fillna("unknown", inplace=True)
data_with_color.shape[0]

528469

In [59]:
##checkpoint to check joined data###
#data_with_color.to_csv('joined_cleaned2019bba.csv',encoding='utf-8')
data_with_color.shape[0]

528469

In [61]:
###FINAL CLEANING OF MODEL ALL COLUMNS 
###cleaning up the year column to have only numbers###
data_with_color['YEAR_BUILT'] = data_with_color['YEAR_BUILT'].str.replace(r'\D+', '')
#data_with_color['YEAR_BUILT'] .replace({'': "unknown"}, inplace=True)
#data_with_color['YEAR_BUILT'].value_counts()

###SELECTING THE FINAL DATA
final_2018 = data_with_color.drop(['viewid','pagePath'],  axis=1)
#complete= data_with_color[data_with_color.YEAR_BUILT.apply(lambda x: x.isnumeric())] 
complete = final_2018[(final_2018['YEAR_BUILT']>'1800') & (final_2018['YEAR_BUILT']<'2022')]
complete = complete[~complete["OEM"].str.contains('\?', na=False)]
#df[(df["A"] >= 1) & (df["B"] < 5)]
complete.shape[0]










<ipython-input-61-48f0acf5feef>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_with_color['YEAR_BUILT'] = data_with_color['YEAR_BUILT'].str.replace(r'\D+', '')


276530

In [62]:
complete.to_csv('20qqq.csv')

By Checking on the error files that had no date, we can see that they are really not relevant and they only carry information of data such as:
1. lead form certificate 
2. Exiting Thank you page
3. Promotional pages (e.g hyundai test drive )
4. Dealer locator link 
all of the link do not contain data that are relevant for analysis and therefore they will be dropped .


In [50]:
#check point to check through in excel##
#final_2018 
complete.to_csv('20timi final_cleaned_.csv',encoding='utf-8')

In [51]:
data_with_color.to_csv('2020a final_cleaned_.csv',encoding='utf-8')